### 线性回归
回归（regression）是能为一个或多个自变量与因变量之间关系建模的一类方法。 在自然科学和社会科学领域，回归经常用来表示输入和输出之间的关系。

输入包含$d$个特征的线性模型预测结果表示为:
$$\hat{y} = w_1 x_1 + ……+ w_dx_d + b$$

将特征向量放到向量$x \in \mathbb{R}^n$中，权重放入向量$w\in\mathbb{R}^d$中，上式可用点乘形式表示:
$$\hat{y} = w^Tx + b$$

用符号表示的矩阵$X \in \mathbb{R}^{n\times d} $其中$X$的每一行是一个样本，每一列是一种特征。线性回归用矩阵形式表示为:
$$\hat{y} = Xw + b$$